# LangChain Basics: Building LLM Applications

LangChain is a framework for developing applications powered by language models. It provides tools to:
- Chain multiple LLM calls together
- Manage prompts and templates
- Handle conversation memory
- Integrate external tools and data sources

## Topics Covered:
1. LangChain architecture and components
2. LLMs and Chat Models
3. Prompt Templates
4. Output Parsers
5. Chains (Sequential, Transform, Router)
6. Memory and Conversation Management
7. Callbacks and Debugging
8. Building a Simple Chatbot

---

## 1. Setup and Installation

In [ ]:
# Install required packages (run once)
# !pip install -U langchain langchain-openai python-dotenv

In [ ]:
import os
import sys
from dotenv import load_dotenv

from langchain_openai import ChatOpenAI

# Prompts live in langchain-core now
from langchain_core.prompts import (
    PromptTemplate,
    ChatPromptTemplate,
    HumanMessagePromptTemplate,
    SystemMessagePromptTemplate,
    FewShotPromptTemplate,
)

# Legacy chains + memory moved to langchain-classic
from langchain_classic.chains import (
    LLMChain,
    SequentialChain,
    SimpleSequentialChain,
    TransformChain,
)
from langchain_classic.memory import (
    ConversationBufferMemory,
    ConversationBufferWindowMemory,
    ConversationSummaryMemory,
)

# Output parsers in langchain-core
from langchain_core.output_parsers import StructuredOutputParser, ResponseSchema

# Callback moved to langchain-community in modern installs
from langchain_community.callbacks.manager import get_openai_callback


load_dotenv()
print('✅ Setup complete!')

ModuleNotFoundError: No module named 'langchain.prompts'

In [7]:
import sys
print(sys.executable)

import langchain
from langchain.prompts import PromptTemplate
print("langchain ok:", langchain.__version__)


/Users/aybikealkan/Library/Caches/pypoetry/virtualenvs/llm-portfolio-fuSmwnhh-py3.12/bin/python


ModuleNotFoundError: No module named 'langchain.prompts'

In [2]:
llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)

resp = llm.invoke("Say 'pong' and nothing else.")
print(resp.content)

NameError: name 'ChatOpenAI' is not defined

## 2. LangChain Models: LLMs vs Chat Models

LangChain supports two types of models:
- **LLM**: Text-in, text-out (legacy completion-style)
- **Chat Model**: Messages-in, message-out (modern)

We'll use Chat Models.

In [ ]:
llm = ChatOpenAI(
    model="gpt-3.5-turbo",
    temperature=0.7,
    max_tokens=150,
)

response = llm.invoke("What is LangChain in one sentence?")
print(response.content)
print('\nType:', type(response))
print('Model:', response.response_metadata.get('model_name', 'N/A'))

In [ ]:
from langchain.schema import HumanMessage, SystemMessage

messages = [
    SystemMessage(content="You are a helpful Python programming tutor."),
    HumanMessage(content="How do I read a CSV file in Python?"),
]

response = llm.invoke(messages)
print(response.content)

## 3. Prompt Templates

### Basic Prompt Template

In [ ]:
template = """You are a {role}.
Answer the following question: {question}
"""

prompt = PromptTemplate(template=template, input_variables=["role", "question"])

formatted_prompt = prompt.format(
    role="data scientist",
    question="What is the difference between correlation and causation?",
)
print(formatted_prompt)
print("=" * 60)

response = llm.invoke(formatted_prompt)
print(response.content)

### Chat Prompt Template

In [ ]:
chat_template = ChatPromptTemplate.from_messages([
    SystemMessagePromptTemplate.from_template(
        "You are a {role} who explains concepts in {style} language."
    ),
    HumanMessagePromptTemplate.from_template("Explain {concept} to me."),
])

messages = chat_template.format_messages(
    role="machine learning expert",
    style="simple",
    concept="gradient descent",
)

for msg in messages:
    print(f"{msg.type}: {msg.content}")

print("=" * 60)
response = llm.invoke(messages)
print(response.content)

### Few-Shot Prompt Template

In [ ]:
examples = [
    {"input": "happy", "output": "sad"},
    {"input": "tall", "output": "short"},
    {"input": "hot", "output": "cold"},
]

example_template = """Input: {input}
Output: {output}"""

example_prompt = PromptTemplate(
    input_variables=["input", "output"],
    template=example_template,
)

few_shot_prompt = FewShotPromptTemplate(
    examples=examples,
    example_prompt=example_prompt,
    prefix="Give the antonym of each word.",
    suffix="Input: {word}\nOutput:",
    input_variables=["word"],
)

formatted = few_shot_prompt.format(word="big")
print(formatted)
print("=" * 60)

response = llm.invoke(formatted)
print(response.content)

## 4. Output Parsers

In [ ]:
response_schemas = [
    ResponseSchema(name="answer", description="The answer to the question"),
    ResponseSchema(name="confidence", description="Confidence level (0-100)"),
    ResponseSchema(name="explanation", description="Brief explanation"),
]

output_parser = StructuredOutputParser.from_response_schemas(response_schemas)
format_instructions = output_parser.get_format_instructions()
print(format_instructions)

In [ ]:
template = """Answer the following question.

{format_instructions}

Question: {question}
"""

prompt = PromptTemplate(
    template=template,
    input_variables=["question"],
    partial_variables={"format_instructions": format_instructions},
)

formatted_prompt = prompt.format(question="What is the capital of France?")
response = llm.invoke(formatted_prompt)
parsed = output_parser.parse(response.content)
print(parsed)

## 5. Chains: Connecting Components

In [ ]:
template = "Tell me a {adjective} fact about {topic}."
prompt = PromptTemplate(template=template, input_variables=["adjective", "topic"])
llm_chain = LLMChain(llm=llm, prompt=prompt)

result = llm_chain.invoke({"adjective": "interesting", "topic": "machine learning"})
print(result["text"])

### Sequential Chains

In [ ]:
template_1 = """You are a creative business consultant.
Generate a company name for a business that makes {product}.
Return only the company name, nothing else.
"""
prompt_1 = PromptTemplate(template=template_1, input_variables=["product"])
chain_1 = LLMChain(llm=llm, prompt=prompt_1)

template_2 = """You are a marketing expert.
Create a catchy tagline for a company called {company_name}.
Return only the tagline, nothing else.
"""
prompt_2 = PromptTemplate(template=template_2, input_variables=["company_name"])
chain_2 = LLMChain(llm=llm, prompt=prompt_2)

sequential_chain = SimpleSequentialChain(chains=[chain_1, chain_2], verbose=True)
result = sequential_chain.invoke("AI-powered code review tools")
print(result["output"])

### Transform Chain (fixed output keys)

In [ ]:
def transform_func(inputs: dict) -> dict:
    text = inputs["text"]
    cleaned = " ".join(text.split()).lower()
    return {"cleaned_text": cleaned}

transform_chain = TransformChain(
    input_variables=["text"],
    output_variables=["cleaned_text"],
    transform=transform_func,
)

summarize_prompt = PromptTemplate(
    template="Summarize this text in one sentence: {cleaned_text}",
    input_variables=["cleaned_text"],
)
summarize_chain = LLMChain(llm=llm, prompt=summarize_prompt, output_key="summary")

full_chain = SequentialChain(
    chains=[transform_chain, summarize_chain],
    input_variables=["text"],
    output_variables=["cleaned_text", "summary"],
    verbose=True,
)

messy_text = """   Machine    Learning    is   a   subset   of
   ARTIFICIAL     INTELLIGENCE    that    enables    computers
   to    LEARN    from    data.   """

result = full_chain.invoke({"text": messy_text})
print("Cleaned:", result["cleaned_text"])
print("Summary:", result["summary"])

## 6. Memory: Maintaining Conversation Context

In [ ]:
memory = ConversationBufferMemory(memory_key="chat_history", input_key="human_input")

template = """The following is a conversation between a human and an AI assistant.
The AI is helpful and friendly.

{chat_history}
Human: {human_input}
AI:"""

prompt = PromptTemplate(template=template, input_variables=["chat_history", "human_input"])
conversation_chain = LLMChain(llm=llm, prompt=prompt, memory=memory, verbose=True)

print(conversation_chain.invoke({"human_input": "Hi, my name is Alice."})["text"])
print(conversation_chain.invoke({"human_input": "What's my name?"})["text"])
print(memory.load_memory_variables({}))

## 7. Callbacks and Cost Tracking

In [ ]:
with get_openai_callback() as cb:
    _ = llm.invoke("What is machine learning?")
    _ = llm.invoke("What is deep learning?")
    _ = llm.invoke("What is the difference between them?")

print("Total Tokens:", cb.total_tokens)
print("Prompt Tokens:", cb.prompt_tokens)
print("Completion Tokens:", cb.completion_tokens)
print("Total Cost (USD):", cb.total_cost)

## 8. Building a Simple Chatbot (fixed memory keys)

In [ ]:
class SimpleChatbot:
    """A simple chatbot with memory and personality."""

    def __init__(self, personality: str = "helpful and friendly", memory_k: int = 5):
        self.llm = ChatOpenAI(temperature=0.7, model="gpt-3.5-turbo")
        self.memory = ConversationBufferWindowMemory(
            k=memory_k,
            memory_key="chat_history",
            input_key="human_input",
        )

        template = f"""You are a {personality} AI assistant.

{{chat_history}}
Human: {{human_input}}
AI:"""

        prompt = PromptTemplate(
            template=template,
            input_variables=["chat_history", "human_input"],
        )

        self.chain = LLMChain(
            llm=self.llm,
            prompt=prompt,
            memory=self.memory,
            verbose=False,
        )

        self.total_cost = 0.0

    def chat(self, message: str) -> str:
        with get_openai_callback() as cb:
            response = self.chain.invoke({"human_input": message})
            self.total_cost += cb.total_cost
        return response["text"]

    def get_cost(self) -> float:
        return self.total_cost

    def reset(self):
        self.memory.clear()
        self.total_cost = 0.0

bot = SimpleChatbot(personality="witty Python programming expert")

conversation_examples = [
    "Hi! I'm learning Python.",
    "What's the difference between a list and a tuple?",
    "Can you show me an example?",
    "Thanks! What should I learn next?",
]

for msg in conversation_examples:
    print("👤 You:", msg)
    print("🤖 Bot:", bot.chat(msg))
    print()

print("Total cost:", bot.get_cost())

## 9. Debugging and Inspection

In [ ]:
template = "Translate '{text}' to {language}."
prompt = PromptTemplate(template=template, input_variables=["text", "language"])
chain = LLMChain(llm=llm, prompt=prompt, verbose=True)

result = chain.invoke({"text": "Hello, how are you?", "language": "Spanish"})
print(result["text"])

## 10. Best Practices Summary

| Memory Type | Use Case | Pros | Cons |
|-------------|----------|------|------|
| Buffer | Short conversations | Simple, complete history | Uses many tokens for long chats |
| Window Buffer | Medium conversations | Limits token usage | Forgets older context |
| Summary | Long conversations | Efficient token usage | Loses some detail |

## 11. Practice Exercises

- Exercise 1: build a 2-step `SequentialChain` that generates 3 questions then answers them.
- Exercise 2: build a sentiment-aware chatbot.
- Exercise 3: build a "code explainer" chain.


## 12. Advanced Patterns Preview

- RAG (Retrieval-Augmented Generation)
- Agents and tool use
- Document loaders + vector stores
- LangSmith monitoring
